## Oversampling doesn't work always and it shouldn't be the first step

In this notebook, we demonstrate how using SMOTE during the training phase of a 5-fold stratified cross-validation leads to an increase in average ROC-AUC and PR-AUC values. This serves as an example that it can enhance performance. However, if you're considering using SMOTE or other oversampling techniques, please bear in mind the following three points::\


*   Implementing these supplementary techniques should not be your initial step in modeling. You need to first take care of data cleaning and preprocessing, feature extration and nomalization if necessary, choosing the right algorithm depending on your data and proper design of testing before attempting to use oversampling techniques.
*   They don't always work as expected. Some machine learning methods, like modeling tabular data with LightGBM, are often reliable, even if not the top choice. However, oversampling methods like SMOTE don't always perform well. How well they work depends on the specific issue and how the data points are spread out.
*   Default settings in Python modules might not always be effective. Similar to any other machine mearnign methodology, SMOTE also has its own hyperparameters. These hyperparameters need to be optimized. For example, in my experience, default SMOTE from `imblearn.over_sampling` never results in better models from performance and confidence perspectives. But choosing right `k_neighbors`, which is one of the hyperparameters of SMOTE helps to make it work, sometimes. In my experience, `k_neighbors` between 2 and 4 are better choices.





We first load Wine dataset from sklearn. We convert the data to a binary classification problem considering category 1 as class positive and the rest of the datapoints as negatives.

In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from imblearn.over_sampling import SMOTE
from sklearn.datasets import load_wine
import warnings

# Load the "Wine" dataset
data = load_wine(as_frame=True)
X = data.data
y = data.target.apply(lambda x: 1 if x == 1 else 0)  # Convert to binary classification: class "1" vs other classes

Then we use 5 fold stratified cross-validation to train and test LightGBM models on Wine dataset and assess their performances in each fold using ROC-AUC and PR-AUC.

In [2]:
# Split the dataset into 5 stratified folds
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

roc_aucs = []
pr_aucs = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Train and test a LightGBM model on each fold
    model = LGBMClassifier(random_state=42, max_depth = 5, verbose = -1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate ROC-AUC and PR-AUC for each fold and save them in lists
    roc_auc = roc_auc_score(y_test, y_pred)
    pr_auc = average_precision_score(y_test, y_pred)

    roc_aucs.append(roc_auc)
    pr_aucs.append(pr_auc)

print(f"ROC-AUCs: {roc_aucs}")
print(f"PR-AUCs: {pr_aucs}")
# Report the mean and variance of the ROC-AUC and PR-AUC across the folds
print(f"Mean ROC-AUC: {np.mean(roc_aucs):.4f}, Variance: {np.var(roc_aucs):.4f}")
print(f"Mean PR-AUC: {np.mean(pr_aucs):.4f}, Variance: {np.var(pr_aucs):.4f}")

ROC-AUCs: [0.9772727272727273, 1.0, 0.9666666666666667, 0.9285714285714286, 0.9642857142857143]
PR-AUCs: [0.9333333333333333, 1.0, 0.9611111111111111, 0.9142857142857143, 0.9571428571428572]
Mean ROC-AUC: 0.9674, Variance: 0.0005
Mean PR-AUC: 0.9532, Variance: 0.0008


We repeat the same training and testing strategy, but this time we use SMOTE with `k_neighbors` of 2.

In [3]:
# Oversampling function
def oversample_smote(X, y, k_neighbours = 2):

  sm = SMOTE(random_state=123, k_neighbors=k_neighbours)
  X_new, y_new = sm.fit_resample(X, y)

  return X_new, y_new

# Split the dataset into 5 stratified folds
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

roc_aucs = []
pr_aucs = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Oversampling using SMOTE
    print(f"Shape of trainign dataframe before applying SMOTE: {X_train.shape}")
    X_train, y_train = oversample_smote(X_train, y_train, k_neighbours = 2)
    print(f"Shape of trainign dataframe after applying SMOTE: {X_train.shape}")
    # Train and test a LightGBM model on each fold
    model = LGBMClassifier(random_state=42, max_depth = 5, verbose = -1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate ROC-AUC and PR-AUC for each fold and save them in lists
    roc_auc = roc_auc_score(y_test, y_pred)
    pr_auc = average_precision_score(y_test, y_pred)

    roc_aucs.append(roc_auc)
    pr_aucs.append(pr_auc)

print(f"ROC-AUCs: {roc_aucs}")
print(f"PR-AUCs: {pr_aucs}")
# Report the mean and variance of the ROC-AUC and PR-AUC across the folds
print(f"Mean ROC-AUC: {np.mean(roc_aucs):.4f}, Variance: {np.var(roc_aucs):.4f}")
print(f"Mean PR-AUC: {np.mean(pr_aucs):.4f}, Variance: {np.var(pr_aucs):.4f}")

Shape of trainign dataframe: (142, 13)
Shape of trainign dataframe: (170, 13)
Shape of trainign dataframe: (142, 13)
Shape of trainign dataframe: (170, 13)
Shape of trainign dataframe: (142, 13)
Shape of trainign dataframe: (172, 13)
Shape of trainign dataframe: (143, 13)
Shape of trainign dataframe: (172, 13)
Shape of trainign dataframe: (143, 13)
Shape of trainign dataframe: (172, 13)
ROC-AUCs: [1.0, 0.9642857142857143, 0.9666666666666667, 1.0, 0.9642857142857143]
PR-AUCs: [1.0, 0.9563492063492064, 0.9611111111111111, 1.0, 0.9571428571428572]
Mean ROC-AUC: 0.9790, Variance: 0.0003
Mean PR-AUC: 0.9749, Variance: 0.0004


Average ROC-AUC and PR-AUC is better when we use SMOTE with `k_neighbors` of 2, in this example. However, performance of each fold didn't increase. When you aim to use SMOTE or any otehr oversampling technique, pay attention to the average performances and perfromance across your test sets, or folds. Also follow the three points provided at the beginning of the notebook.